# Задание 1

Будем парсить страницу со свежими новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [44]:
import requests

from bs4 import BeautifulSoup

import time

In [89]:
URL = 'https://habr.com/ru/search/'
KEYWORDS = ['python', 'парсинг']

def get_all_links(url, query, pages):
    link_list = []
    for q in query:
        params = {
            'q': q,
            'target_type': 'posts'
        }
        headers = {
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
        }
        for i in range(1, pages):
            req = requests.get(URL + 'page' + str(i), params=params, headers=headers)
            time.sleep(0.03)
            soup = BeautifulSoup(req.text, 'html.parser')
            # добираемся до блоков с новостями
            news_blocks = soup.find_all('article', class_="post post_preview")
            # добираемся до текста со ссылкой
            articles_intro = list(map(lambda x: x.find('h2', class_='post__title'), news_blocks))
            # добираемся до ссылок
            link_list += list(map(lambda x: x.find('a', class_="post__title_link").get('href'), articles_intro))

    return link_list

all_links = get_all_links(URL, KEYWORDS, 50)

160

In [114]:
import pandas as pd

In [90]:
def get_habr_news(links):
    habr_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text)
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
        title = soup.find('h1', class_='post__title post__title_full').text
        text = soup.find('div', class_='post__body post__body_full').text
        row = {'date': date, 'title': title, 'link': link, 'text': text}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
    return habr_news

habr_news = get_habr_news(all_links)
habr_news

,date,title,link,text
0,2020-04-21,"\nВышел Python 2.7.18, последний релиз ветки P...",https://habr.com/ru/news/t/498364/,"\n\r\n20 апреля 2020 года, спустя почти десять..."
0,2020-12-04,\nPython как компилируемый статически типизиро...,https://habr.com/ru/news/t/531402/,\nПо данным широко известного в узких кругах T...
0,2020-03-03,\nВ начале этого года Python сместил Java и ст...,https://habr.com/ru/company/itsumma/news/t/490...,"\nСогласно отчету RedMonk за январь 2020 года,..."
0,2020-06-03,\nВышла версия 0.0.2 snakeware — дистрибутива ...,https://habr.com/ru/news/t/505096/,\n\n\r\n31 мая 2020 года разработчик Джош Мур ...
0,2020-11-13,\nСоздатель Python Гвидо ван Россум выходит на...,https://habr.com/ru/news/t/527858/,\n\n\r\nСоздатель языка программирования Pytho...
...,...,...,...,...
0,2019-09-12,\nPHP-библиотеки для e-commerce: работа с АТОЛ...,https://habr.com/ru/company/lamoda/blog/466039/,"\nПривет, меня зовут Павел Савельев, я руковод..."
0,2021-01-15,\nИспользуйте парсинг вместо контроля типов\n,https://habr.com/ru/company/vdsina/blog/537398/,\n\n\r\nВ 2019 году была написана потрясающая ...
0,2014-04-21,\nОсновы парсинга с помощью Python+lxml\n,https://habr.com/ru/post/220125/,"\nДобрый день, уважаемые читатели.\r\nВ сегодн..."
0,2011-03-09,\nПростая библиотека для парсинга HTML\n,https://habr.com/ru/post/115135/,"\nНедавно зарелизил Leaf, это небольшая библио..."


# Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [131]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:85.0) Gecko/20100101 Firefox/85.0',

    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',

    'Accept': 'application/json, text/plain, */*',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/hackcheck/'
}

In [133]:
EMAILS = ['xxx@x.ru', 'yyy@y.com']

def data_from_json(json_file, mail):
    df_full = pd.DataFrame()
    for id_, data in json_file['breaches'].items():
        row = {}
        row['email'] = mail
        row['date'] = data['publishDate']
        row['site'] = data['site']
        row['description'] = data['description']
        df_full = pd.concat([df_full, pd.DataFrame([row])])
    return df_full

def check_emails(emails):
    checked_emails = pd.DataFrame()
    for email in emails:
        PAYLOAD = {
            "emailAddresses":[email]
        }
        r = requests.post(URL, headers=HEADERS, json=PAYLOAD)
        row = data_from_json(r.json(), email)
        checked_emails = pd.concat([checked_emails, row])
    return checked_emails

check_emails(EMAILS)

,email,date,site,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
